# GANomaly Notebook Results

## Initial Configurations

### Selecting the device to work with

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

### Libraries import

In [2]:
import sys
import random
import numpy as np
import tensorflow as tf
from IPython.display import clear_output

sys.path.append("../")

In [3]:
from datasets.dict_features import get_parkinson
from utils.metrics import get_true_positives, get_true_negatives, get_false_positives, get_false_negatives
from utils.metrics import accuracy, precision, recall, specificity, f1_score, get_AUC
from utils.metrics import shapiroWilks_test, dagostinoPearson_test, bartlett_test, levene_test, fOneWay_test
from utils.savers import save_videos, save_latent_vectors

from models.ganomaly.model import get_2D_models, get_3D_models
from models.ganomaly.utils.losses import l1_loss, l2_loss, BCELoss, l1_loss_batch, l2_loss_batch
from models.ganomaly.utils.preprocessing import normalize_accros_channels, min_max_scaler, resize
from models.ganomaly.utils.preprocessing import get_center_of_volume, delete_patient_id
from models.ganomaly.utils.weights_init import reinit_model
from models.ganomaly.utils.exp_docs import experiment_folder_path, get_metrics_path, get_outputs_path
from models.ganomaly.utils.printers import print_metrics
from models.ganomaly.utils.savers import save_errors

### GPU Memory Configuration

In [4]:
if os.getenv("CUDA_VISIBLE_DEVICES") != '-1':
    gpus = tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(gpus[0], True)
tf.debugging.set_log_device_placement(False)

## Dataset Processing

### Data loading

In [ ]:
N_CPUS = 16
dataset_path = "/data/Datasets/parkinson/tf_records/data2020_cuttedFrames/parkinson_2020_cutted.tfrecord"
encoding_dictionary = get_parkinson()
encoding_dictionary

### General extraction function for tfrecords

In [ ]:
def from_bytes_to_dict(example_bytes, encoding_dictionary):
    return tf.io.parse_single_example(example_bytes, encoding_dictionary)

def extract_data_from_dict(example_dict):
    f = example_dict["frames"]
    h = example_dict["height"]
    w = example_dict["width"]
    c = example_dict["channels"]
    raw_volume = tf.io.decode_raw(example_dict["video"], tf.uint8)
    volume = tf.reshape(raw_volume, [f,h,w,c])
    return tf.cast(volume, dtype=tf.float32), example_dict["parkinson"], example_dict["id"]

### Data pipeline

In [ ]:
raw_data = tf.data.TFRecordDataset(dataset_path)
dict_data = raw_data.map(lambda x: from_bytes_to_dict(x, encoding_dictionary), N_CPUS)
total_data = dict_data.map(extract_data_from_dict, N_CPUS)
total_data

In [ ]:
shape_videos = []
labels_videos = []
patients_ids = []
for x, y, z in total_data:
    shape_videos.append(x.numpy().shape)
    labels_videos.append(y.numpy())
    patients_ids.append(z.numpy())
shape_videos = np.r_[shape_videos]
labels_videos = np.r_[labels_videos]
patients_ids = np.r_[patients_ids]
print("Data information about the data")
print("Total videos: ", shape_videos.shape[0])
print("Min value of frames: ", np.min(shape_videos[:,0]))
print("Max value of frames: ", np.max(shape_videos[:,0]))
print("Mean value of frames: ", np.mean(shape_videos[:,0]))
print("Unique ids: ", np.unique(patients_ids, return_counts=True))

In [ ]:
normal_data = total_data.filter(lambda x,y,z: tf.equal(y, 0))
abnormal_data = total_data.filter(lambda x,y,z: tf.equal(y, 1))

normal_patients = []
for i in np.unique(patients_ids):
    normal_patients.append(normal_data.filter(
            lambda x, y, z: tf.equal(z, i)
        ).map(
            delete_patient_id, N_CPUS
        ).map(
            lambda x, y: resize(x, y, [64,64]), N_CPUS
        ).map(
            lambda x, y: get_center_of_volume(x, y, 64), N_CPUS
        ).map(
            lambda x, y: normalize_accros_channels(x, y, 0.5, 0.5), N_CPUS
        ).map(
            lambda x, y: min_max_scaler(x, y, -1., 1.), N_CPUS
        ).cache()
    )

abnormal_data = abnormal_data.map(
    delete_patient_id, N_CPUS
).map(
    lambda x, y: resize(x, y, [64,64]), N_CPUS
).map(
    lambda x, y: get_center_of_volume(x, y, 64), N_CPUS
).map(
    lambda x, y: normalize_accros_channels(x, y, 0.5, 0.5), N_CPUS
).map(
    lambda x, y: min_max_scaler(x, y, -1., 1.), N_CPUS
).cache()

normal_patients, abnormal_data

In [ ]:
# from IPython.display import display, clear_output
# import matplotlib.pyplot as plt
# from ipywidgets import interact, IntSlider

# x = x.numpy().astype(np.uint8)

# interact(lambda frame: plt.imshow(x[frame-1]), frame=IntSlider(min=1, max=x.shape[0], step=1));

## Model

### Model params

In [5]:
isize = 64 # Input size of the data (image or volume)
nz = 100 # Context vector size
nc = 3 # Quantity of channels in the data
ngf = 64 # Quantity of initial filters in the first convolution of the encoder
extra_layers = 0 # Quantity of layer blocks to add before reduction
w_gen = (1, 50, 1) # Tuple with 3 elements (w_adv, w_con, w_enc) to use in the error of generator

### Experiment params

In [6]:
model_dimension = "3D" # Dimension of model to use in experiment
batch_size = 16 # Size of the bath for the model
epochs = 30000 # Quantity of epochs to do in the training
beta_1 = 0.5 # Momentum of beta 1 in adam optimizer for generator and discriminator
beta_2 = 0.999 # Momentum of beta 2 in adam optimizer for generator and discriminator
lr = 0.0002 # Initial learning rate for adam optimizer

### Replicability configuration

In [7]:
random.seed(8128)
np.random.seed(8128)
tf.random.set_seed(8128)

### Experiment documentation

In [9]:
experiment_path, experiment_id = experiment_folder_path("/home/jefelitman/Saved_Models/Anomaly_Parkinson/", model_dimension, isize, nc)

# Metrics folder for model graphs
metric_save_path = get_metrics_path(experiment_path)

# Output folder for outputs
outputs_path = get_outputs_path(experiment_path)
experiment_path, metric_save_path, outputs_path

('/home/jefelitman/Saved_Models/Anomaly_Parkinson/0001_Ganomaly_3D-64x64x64x3',
 '/home/jefelitman/Saved_Models/Anomaly_Parkinson/0001_Ganomaly_3D-64x64x64x3/metrics',
 ['/home/jefelitman/Saved_Models/Anomaly_Parkinson/0001_Ganomaly_3D-64x64x64x3/outputs/latent_vectors/in',
  '/home/jefelitman/Saved_Models/Anomaly_Parkinson/0001_Ganomaly_3D-64x64x64x3/outputs/latent_vectors/out',
  '/home/jefelitman/Saved_Models/Anomaly_Parkinson/0001_Ganomaly_3D-64x64x64x3/outputs/samples/real',
  '/home/jefelitman/Saved_Models/Anomaly_Parkinson/0001_Ganomaly_3D-64x64x64x3/outputs/samples/fake',
  '/home/jefelitman/Saved_Models/Anomaly_Parkinson/0001_Ganomaly_3D-64x64x64x3/outputs/samples/substraction',
  '/home/jefelitman/Saved_Models/Anomaly_Parkinson/0001_Ganomaly_3D-64x64x64x3/outputs/errors/adversarial',
  '/home/jefelitman/Saved_Models/Anomaly_Parkinson/0001_Ganomaly_3D-64x64x64x3/outputs/errors/contextual',
  '/home/jefelitman/Saved_Models/Anomaly_Parkinson/0001_Ganomaly_3D-64x64x64x3/outputs/e

In [ ]:
readme = open(os.path.join(experiment_path, "README.txt"), "w+")
readme.write(
"""This file contains information about the experiment made in this instance.

All models saved don't include the optimizer, but this file explains how to train in the same conditions.

Basic notation:
- {i}_Ganomaly_{d}: Experiment id, name of the model and operation dimensionality of convolutions.
- H x W x C or F x H x W x C: Data dimensions used where F are frames, H height, W width and C channels.

Experiment settings:
- The seed used was 8128 for python random module, numpy random and tf random after the library importations.
- The batch size was of {batch}.
- The optimizer used in this experiment was Adam for generator and discriminator.
- The number of classes in this dataset are 2 (Normal and Parkinson) .
- This experiment use the data of parkinson_2020_cutted.tfrecord from data2020_cuttedFrames data.
- The initial lr was of {lr}.
- The beta 1 and beta 2 for adam optimizer was {beta_1} and {beta_2} respectively.
- The total epochs made in this experiment was of {epochs}.
- The context vector size (nz) was of {nz}.
- The # channels in data (nc) was of {nc}.
- The initial filters in the first convolution of the encoder was {ngf}.
- The quantity of layer blocks to add before reduction was of {extra_layers}.
- The weights for adversarial, contextual and encoder error in generator was {w_gen}.

Transformations applied to data (following this order):
- Resize: We resize the frames of volumes to H x W (64 x 64).
- Centered volume: We take 64 frames on the center of volume to train and test the data.
- Normalize: We normalize the volume with mean and std of 0.5 for both.
- Scale: We scale the data between -1 and 1 using min max scaler to be comparable with generated images.
- Randomize: We randomize the order of samples in every epoch.

Training process:
- The data doesn't have train and test partition but we make the partitions like this:
    * 81.8% (72 videos/9 patients) of normal data is used in train randomly selected.
    * 29.2% (16 videos/2 patients) of normal data is used in test randomly selected.
    * 100% of abnormal (parkinson) data is used in test.
""".format(
        i = experiment_id,
        d = model_dimension,
        batch = batch_size,
        lr = lr,
        beta_1 = beta_1,
        beta_2 = beta_2,
        epochs = epochs,
        nz = nz,
        nc = nc,
        ngf = ngf,
        extra_layers = extra_layers,
        w_gen = w_gen
    )
)
readme.close()

### Models creation

In [ ]:
gen_model, disc_model = globals()["get_{}_models".format(
    model_dimension
)](isize, nz, nc, ngf, extra_layers)
gen_model.summary()
disc_model.summary()

### Optimizers creation

In [ ]:
gen_opt = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=beta_1, beta_2=beta_2)
disc_opt = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=beta_1, beta_2=beta_2)
gen_opt, disc_opt

### Train and Inference steps

In [ ]:
# %load '../models/ganomaly/utils/steps.py'
"""This file contains the different steps (training and testing) for GANomaly nets in Tensorflow.
https://arxiv.org/abs/1805.06725
Version: 1.0
Made by: Edgar Rangel
"""

@tf.function
def train_step(x_data, w_gen = (1, 50, 1)):
    """Function that make one train step for whole GANomaly model and returns the errors and 
    relevant output variables.
    Dependencies:
        gen_model: A variable instance (with this name) of the generator model to be trained. (Keras Model Instance).
        gen_opt: A variable instance (with this name) of optimizer for generator model to apply the learning process (Keras Optimizers Instance).
        disc_model: A variable instance (with this name) of the discriminator model to be trained. (Keras Model Instance).
        disc_opt: A variable instance (with this name) of optimizer for discriminator model to apply the learning process (Keras Optimizers Instance).
    Args:
        x_data: A Tensor with the batched data to be given for the model in the step (Tensor Instance).
        w_gen: An instance of tuple with 3 elements in the following order (w_adv, w_con, w_enc) to use in the error of generator (Tuple).
    """
    assert len(w_gen) == 3
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        #Forward process of networks
        fake, latent_i, latent_o = gen_model(x_data, training=True)
        pred_real, feat_real = disc_model(x_data, training=True)
        pred_fake, feat_fake = disc_model(fake, training=True)
        #Losses compute for generator
        err_g_adv = l2_loss(feat_fake, feat_real)
        err_g_con = l1_loss(x_data, fake)
        err_g_enc = l2_loss(latent_i, latent_o)
        err_g = err_g_adv * w_gen[0] + err_g_con * w_gen[1] +  err_g_enc * w_gen[2]
        #Losses compute for discriminator
        err_d_real = BCELoss(tf.ones_like(pred_real), pred_real)
        err_d_fake = BCELoss(tf.zeros_like(pred_fake), pred_fake)
        err_d = (err_d_real + err_d_fake) * 0.5

    gradients_g = gen_tape.gradient(err_g, gen_model.trainable_variables)
    gradients_d = disc_tape.gradient(err_d, disc_model.trainable_variables)

    gen_opt.apply_gradients(zip(gradients_g, gen_model.trainable_variables))
    disc_opt.apply_gradients(zip(gradients_d, disc_model.trainable_variables))

    return err_g, err_d, fake, latent_i, latent_o, feat_real, feat_fake

@tf.function
def test_step(x_data):
    """Function that make one inference step for whole GANomaly model and returns its outputs to evaluate them.
    Dependencies:
        gen_model: A variable instance (with this name) of the generator model to be trained. (Keras Model Instance).
    Args:
        x_data: A Tensor with the batched data to be given for the model in the step (Tensor Instance).
    """
    fake, latent_i, latent_o = gen_model(x_data, training=False)
    pred_real, feat_real = disc_model(x_data, training=False)
    pred_fake, feat_fake = disc_model(fake, training=False)
    return fake, latent_i, latent_o, feat_real, feat_fake

## Training process

### Metrics creation

In [ ]:
TP = get_true_positives()
TN = get_true_negatives()
FP = get_false_positives()
FN = get_false_negatives()
AUC = get_AUC()

train_metrics_csv = open(os.path.join(metric_save_path,"train.csv"), "w+")
train_metrics_csv.write("epoch, gen_error, disc_error, accuracy, precision, recall, specificity, f1_score, auc\n")

test_metrics_csv = open(os.path.join(metric_save_path,"test.csv"), "w+")
test_metrics_csv.write("epoch, accuracy, precision, recall, specificity, f1_score, auc\n")

### Loop

In [ ]:
for epoch in range(epochs):
    # Data partition for train and test
    partition_point = len(normal_patients) - 2
    np.random.shuffle(normal_patients)
    train_data = normal_patients[0]
    for i in range(1, partition_point):
        train_data = train_data.concatenate(normal_patients[i])
    train_data = train_data.shuffle(72, reshuffle_each_iteration=True).batch(batch_size).prefetch(-1)
    
    test_data = normal_patients[partition_point]
    for i in range(partition_point+1, len(normal_patients)):
        test_data = test_data.concatenate(normal_patients[i])
    test_data = test_data.concatenate(abnormal_data)
    test_data = test_data.shuffle(88+16, reshuffle_each_iteration=True).batch(batch_size).prefetch(-1)
    
    for step, xy in enumerate(train_data):
        err_g, err_d, fake_images, latent_i, latent_o, feat_real, feat_fake = train_step(xy[0])
        
        if err_d < 1e-5:
            reinit_model(disc_model)
            
        anomaly_scores = tf.math.reduce_mean(tf.math.pow(tf.squeeze(latent_i-latent_o), 2), axis=1)
        anomaly_scores = (anomaly_scores - tf.reduce_min(anomaly_scores)) / (
            tf.reduce_max(anomaly_scores) - tf.reduce_min(anomaly_scores)
        )
            
        TP.update_state(xy[1], anomaly_scores)
        TN.update_state(xy[1], anomaly_scores)
        FP.update_state(xy[1], anomaly_scores)
        FN.update_state(xy[1], anomaly_scores)
        AUC.update_state(xy[1], anomaly_scores)
        acc = accuracy(TP.result().numpy(), TN.result().numpy(), FP.result().numpy(), FN.result().numpy())
        pre = precision(TP.result().numpy(), FP.result().numpy())
        rec = recall(TP.result().numpy(), FN.result().numpy())
        spe = specificity(TN.result().numpy(), FP.result().numpy())
        f1 = f1_score(TP.result().numpy(), FP.result().numpy(), FN.result().numpy())
        auc = AUC.result().numpy()
        
        clear_output(wait=True)
        print_metrics(epoch, step, acc, pre, rec, spe, f1, auc, err_g, err_d)
        
        # Save the latent vectors, videos and errors in the last epoch
        if epoch + 1 == epochs:
            save_latent_vectors(tf.squeeze(latent_i).numpy(), xy[1].numpy(), outputs_path[0], True)
            save_latent_vectors(tf.squeeze(latent_o).numpy(), xy[1].numpy(), outputs_path[1], True)
            
            batch_videos = min_max_scaler(xy[0], 0, 0, 255)[0].numpy()
            save_videos(batch_videos, xy[1].numpy(), outputs_path[2], model_dimension, True)
            batch_videos = min_max_scaler(fake_images, 0, 0, 255)[0].numpy()
            save_videos(batch_videos, xy[1].numpy(), outputs_path[3], model_dimension, True)
            batch_videos = min_max_scaler(tf.abs(xy[0] - fake_images), 0, 0, 255)[0].numpy()
            save_videos(batch_videos, xy[1].numpy(), outputs_path[4], model_dimension, True)
            
            save_errors(l2_loss_batch(feat_real, feat_fake), xy[1].numpy(), outputs_path[5], True)
            save_errors(l1_loss_batch(xy[0], fake_images), xy[1].numpy(), outputs_path[6], True)
            save_errors(l2_loss_batch(latent_i, latent_o), xy[1].numpy(), outputs_path[7], True)
        
    # Save train metrics
    train_metrics_csv.write("{e}, {loss_g}, {loss_d}, {acc}, {pre}, {rec}, {spe}, {f1}, {auc}\n".format(
        e = epoch,
        loss_g = err_g,
        loss_d = err_d,
        acc = acc,
        pre = pre,
        rec = rec,
        spe = spe,
        f1 = f1,
        auc = auc
    ))
    TP.reset_states()
    TN.reset_states()
    FP.reset_states()
    FN.reset_states()
    AUC.reset_states()
    
    for step, xy in enumerate(test_data):
        fake_images, latent_i, latent_o, feat_real, feat_fake = test_step(xy[0])
        
        anomaly_scores = tf.math.reduce_mean(tf.math.pow(tf.squeeze(latent_i-latent_o), 2), axis=1)
        anomaly_scores = (anomaly_scores - tf.reduce_min(anomaly_scores)) / (
            tf.reduce_max(anomaly_scores) - tf.reduce_min(anomaly_scores)
        )
            
        TP.update_state(xy[1], anomaly_scores)
        TN.update_state(xy[1], anomaly_scores)
        FP.update_state(xy[1], anomaly_scores)
        FN.update_state(xy[1], anomaly_scores)
        AUC.update_state(xy[1], anomaly_scores)
        acc = accuracy(TP.result().numpy(), TN.result().numpy(), FP.result().numpy(), FN.result().numpy())
        pre = precision(TP.result().numpy(), FP.result().numpy())
        rec = recall(TP.result().numpy(), FN.result().numpy())
        spe = specificity(TN.result().numpy(), FP.result().numpy())
        f1 = f1_score(TP.result().numpy(), FP.result().numpy(), FN.result().numpy())
        auc = AUC.result().numpy()
        
        clear_output(wait=True)
        print_metrics(epoch, step, acc, pre, rec, spe, f1, auc)
        
        # Save the latent vectors, videos and errors in the last epoch
        if epoch + 1 == epochs:
            save_latent_vectors(tf.squeeze(latent_i).numpy(), xy[1].numpy(), outputs_path[0], False)
            save_latent_vectors(tf.squeeze(latent_o).numpy(), xy[1].numpy(), outputs_path[1], False)
            
            batch_videos = min_max_scaler(xy[0],0,0,255)[0].numpy()
            save_videos(batch_videos, xy[1].numpy(), outputs_path[2], model_dimension, False)
            batch_videos = min_max_scaler(fake_images,0,0,255)[0].numpy()
            save_videos(batch_videos, xy[1].numpy(), outputs_path[3], model_dimension, False)
            batch_videos = min_max_scaler(tf.abs(xy[0] - fake_images), 0, 0, 255)[0].numpy()
            save_videos(batch_videos, xy[1].numpy(), outputs_path[4], model_dimension, False)
            
            save_errors(l2_loss_batch(feat_real, feat_fake), xy[1].numpy(), outputs_path[5], False)
            save_errors(l1_loss_batch(xy[0], fake_images), xy[1].numpy(), outputs_path[6], False)
            save_errors(l2_loss_batch(latent_i, latent_o), xy[1].numpy(), outputs_path[7], False)
        
    # Save test metrics
    test_metrics_csv.write("{e}, {acc}, {pre}, {rec}, {spe}, {f1}, {auc}\n".format(
        e = epoch,
        acc = acc,
        pre = pre,
        rec = rec,
        spe = spe,
        f1 = f1,
        auc = auc
    ))
    TP.reset_states()
    TN.reset_states()
    FP.reset_states()
    FN.reset_states()
    AUC.reset_states()
    
train_metrics_csv.close()
test_metrics_csv.close()

### Save models

In [ ]:
gen_model.save(os.path.join(experiment_path,"gen_model.h5"), include_optimizer=False, save_format='h5')
disc_model.save(os.path.join(experiment_path,"disc_model.h5"), include_optimizer=False, save_format='h5')

In [ ]:
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

x = min_max_scaler(fake_images[1], 0, 0, 1)[0].numpy()

interact(lambda frame: plt.imshow(x[frame-1]), frame=IntSlider(min=1, max=x.shape[0], step=1));

In [ ]:
x_rela = min_max_scaler(xy[0][1], 0, 0, 1)[0].numpy()

interact(lambda frame: plt.imshow(x_rela[frame-1]), frame=IntSlider(min=1, max=x_rela.shape[0], step=1));